<a href="https://colab.research.google.com/github/nicolaiberk/bild/blob/main/code/content_analysis/bert/mig/_mig_clsfr/BERT_estimates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers datasets

In [3]:
import os
import csv
import sys
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import islice
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

csv.field_size_limit(sys.maxsize)

131072

In [4]:
# get full set of news articles
if not os.path.isfile('newspapers/_bild_articles.csv'):
    os.system('wget -O articles.zip https://www.dropbox.com/sh/bbf0655w9931xbk/AADQNpkipxBENPk4Gp5j1UaDa?dl=0')
    os.system('unzip articles.zip -d newspapers')
    os.system('rm articles.zip')

In [5]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:
model_name = "drive/MyDrive/Bild/mig_clsfr_BERT_torch"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to(device)  # Move model to GPU
model.eval()  # Set model to evaluation mode
tokenizer = AutoTokenizer.from_pretrained('bert-base-german-cased', max_length = 512, truncation=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


In [7]:
from datasets import load_dataset

In [8]:
paper = os.listdir('newspapers')[3]
paper

'_bild_articles_2019.csv'

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [10]:
filename = 'newspapers/'+paper
print(f'Processing file {paper}')
dataset = load_dataset("csv", data_files=filename, split="train", streaming=True)

Processing file _bild_articles_2019.csv


In [11]:
def infer(batch):
    with torch.no_grad():
        inputs = tokenizer(batch["text"], padding=True, truncation=True, max_length=512, return_tensors="pt")
        inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU
        outputs = model(**inputs)  # Model inference on GPU
        predictions = torch.argmax(outputs.logits, dim=-1)
        return predictions

In [12]:
## remove empty texts
dataset = dataset.filter(lambda example: example["text"] is not None)

In [13]:
batch_size = 64

for paper in os.listdir('newspapers'):
  filename = 'newspapers/'+paper
  print(f'Processing file {paper}')
  with open(filename, 'r') as csvfile:
    with open('drive/MyDrive/Bild/BERT_estimates.csv', mode='a') as fo:
      reader = csv.reader(csvfile)
      for row in reader:
          # define relative position in row based on title
          titlerow = np.argmax([r == 'title' for r in row])
          linkrow  = np.argmax([r == 'url'   for r in row])
          daterow  = np.argmax([r == 'date'  for r in row])
          textrow  = np.argmax([r == 'text'  for r in row])
          topicrow = np.argmax([r == 'topic' for r in row])
          break

      writer = csv.writer(fo)
      writer.writerow(['title', 'link', 'date', 'topic', 'est', 'label'])

      titlebatch = []
      linkbatch = []
      datebatch = []
      textbatch = []
      topicbatch = []

      count = 0
      batches_run = 0

      for row in tqdm(reader):
        if count >= batch_size:
            estbatch = infer(textbatch)

            for title, link, date, topic, est, label in zip(titlebatch, linkbatch, datebatch, topicbatch, estbatch, labelbatch):
              writer.writerow([title, link, date, topic, est, label])

            count = 0
            titlebatch = []
            linkbatch = []
            datebatch = []
            textbatch = []
            topicbatch = []

        titlebatch.append(row[titlerow])
        linkbatch.append(row[linkrow])
        datebatch.append(row[daterow])
        textbatch.append(row[textrow])
        topicbatch.append(row[topicrow])
        count += 1

      if len(titlebatch) > 0:
        for title, link, date, topic, est, label in zip(titlebatch, linkbatch, datebatch, topicbatch, estbatch, labelbatch):
              writer.writerow([title, link, date, topic, est, label])
    print(f'\tFinished file {paper}.')

Processing file _sz_articles_2019.csv


0it [00:00, ?it/s]


NameError: name 'batch_size' is not defined